In [2]:
import os
import json
import random
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModel
import pdb
import string
import fairseq

#### Set up OpenAI API

In [4]:
import openai

openai.api_key = '<YOUR OPENAI API KEY>'

def call_gpt(cur_prompt, stop, temperature = 0):
    ans = openai.Completion.create(
                model="gpt-3.5-turbo-instruct",
                max_tokens=256,
                stop=stop,
                prompt=cur_prompt,
                temperature=temperature)
    returned = ans['choices'][0]['text']
    
    return returned

#### Functions for retrieval models (Contriever)

In [5]:
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def get_sent_embeddings(sents, contriever, tok, BSZ=32):    
    all_embs = []
    for i in range(0, len(sents), BSZ):
        sent_batch = sents[i:i+BSZ]
        inputs = tok(sent_batch, padding=True, truncation=True, return_tensors='pt').to("cuda")
        with torch.no_grad():
            outputs = contriever(**inputs)
            embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
        all_embs.append(embeddings.cpu())
    all_embs = torch.vstack(all_embs)
    return all_embs

def retrieve_facts(query, fact_embs, contriever, tok, k=2):
    inputs = tok([query], padding=True, truncation=True, return_tensors='pt').to("cuda")
    with torch.no_grad():
        outputs = contriever(**inputs)
        query_emb = mean_pooling(outputs[0], inputs['attention_mask']).cpu()
    sim = (query_emb @ fact_embs.T)[0]
    knn = sim.topk(min(len(fact_embs), k), largest=True)
    return knn.indices

#### Load dataset

In [6]:
with open('datasets/MQuAKE-CF-3k.json', 'r') as f:
    dataset = json.load(f)

#### Build a memory index which contains all the edits

In [7]:
new_facts = set()
for d in dataset:
    for r in d["requested_rewrite"]:
        new_facts.add(f'{r["prompt"].format(r["subject"])} {r["target_new"]["str"]}')
new_facts = list(new_facts)
len(new_facts)

2786

In [8]:
contriever = AutoModel.from_pretrained("facebook/contriever-msmarco").cuda()
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever-msmarco")

In [9]:
embs = get_sent_embeddings(new_facts, contriever, tokenizer)

In [10]:
# Run test for retrieval index
fact_ids = retrieve_facts("The U.S. president is ", embs, contriever, tokenizer)
print(new_facts[fact_ids[0]])

The name of the current head of state in United States of America is Norodom Sihamoni


#### Run DeepEdit

In [13]:
# read prompts
with open('prompts/multihop-cot-prompts.txt', 'r') as f:
    task_prompt = f.read()

with open('prompts/delete-prompt.txt', 'r') as f:
    delete_prompt = f.read()
    
with open('prompts/conflict-prompt.txt', 'r') as f:
    conflict_prompt = f.read()

with open('prompts/conflict-prompt-1.txt', 'r') as f:
    conflict_prompt_1 = f.read()
    
with open('prompts/conflict-prompt-2.txt', 'r') as f:
    conflict_prompt_2 = f.read()
    
with open('prompts/entity-prompt.txt', 'r') as f:
    entity_prompt = f.read()
    
with open('prompts/exist-prompt.txt', 'r') as f:
    exist_prompt = f.read()

def searchFact(sentence_1, sentence_2, fact_ls):
    embs = get_sent_embeddings(fact_ls, contriever, tokenizer)
    
    prompt_1 = delete_prompt + "\n\nSentence A: " + sentence_1 + '\nSentence B: ' + sentence_2 + '\nSentence B after removal: '
    gen = call_gpt(prompt_1, ['\n']).strip()
    thought_for_search = gen

    fact_ids = retrieve_facts(thought_for_search, embs, contriever, tokenizer)
    fact_sent_ls = [fact_ls[id_] + '.' for id_ in fact_ids]
    return fact_sent_ls

def newEntity(sentence_1, sentence_2):
    prompt_1 = entity_prompt + "\n\nSentence A: " + sentence_1 + '\nSentence B: ' + sentence_2 + '\nAnswer: '
    gen = call_gpt(prompt_1, ['\n']).strip()
    return gen

def entityExist(entity, sentence):
    if entity.lower() in sentence.lower():
        return True
    prompt_4 = exist_prompt + "\n\nSentence: " + sentence + '\nEntity: ' + entity + '\nAnswer: '
    gen = call_gpt(prompt_4, ['\n']).strip().strip(string.punctuation)
    for i_ in range(2): 
        if gen.lower() == 'yes' or gen.lower() == 'no':
            break
        gen = call_gpt(prompt_4, ['\n'], temperature = 1).strip().strip(string.punctuation)
    
    if gen.lower() == 'yes':
        return True
    else:
        return False
    
def judgeConflict1(sentence_1, sentence_2):
    prompt_2 = conflict_prompt_1 + "\n\nSentence A: " + sentence_1 + '\nSentence B: ' + sentence_2 + '\nAnswer: '
    gen = call_gpt(prompt_2, ['\n']).strip().strip(string.punctuation)
    for i_ in range(2): 
        if gen.lower() == 'yes' or gen.lower() == 'no':
            break
        
        gen = call_gpt(prompt_2, ['\n'], temperature = 1).strip().strip(string.punctuation)
    
    if gen.lower() == 'yes':
        return True
    else:
        return False
    
def judgeConflict2(sentence_1, sentence_2):
    prompt_2 = conflict_prompt_2 + "\n\nSentence A: " + sentence_1 + '\nSentence B: ' + sentence_2 + '\nAnswer: '
    gen = call_gpt(prompt_2, ['\n']).strip().strip(string.punctuation)
    for i_ in range(2): 
        if gen.lower() == 'yes' or gen.lower() == 'no':
            break
        
        gen = call_gpt(prompt_2, ['\n'], temperature = 1).strip().strip(string.punctuation)
    
    if gen.lower() == 'yes':
        return True
    else:
        return False
    
def judgeConflict(sentence_1, sentence_2):
    prompt_2 = conflict_prompt + "\n\nSentence A: " + sentence_1 + '\nSentence B: ' + sentence_2 + '\nAnswer: '
    gen = call_gpt(prompt_2, ['\n']).strip().strip(string.punctuation)
    for i_ in range(2): 
        if gen.lower() == 'yes' or gen.lower() == 'no':
            break
        gen = call_gpt(prompt_2, ['\n'], temperature = 1).strip().strip(string.punctuation)
    
    if gen.lower() == 'yes':
        return True
    else:
        return False
    
def oneStep(prompt, last_entity, new_entity, last_thought, count):
    gen = call_gpt(prompt, ['.# ', '.\n']).strip()
    if 'the answer' in gen or 'the answers' in gen:
        return new_entity
    current_thought = (gen + '.').strip()

    fact_sent_ls = searchFact(last_thought, current_thought, fact_ls)

    output_ls = []
    for fact_sent_ in fact_sent_ls[::-1]:
        last_entity_flag = entityExist(last_entity, fact_sent_)
        exist_flag = entityExist(new_entity, fact_sent_)
        conflict_flag = judgeConflict(fact_sent_, current_thought) and judgeConflict1(fact_sent_, current_thought) and judgeConflict2(fact_sent_, current_thought)
        if (exist_flag and not last_entity_flag) or conflict_flag:
            new_thought = fact_sent_
            output = {
                'last_entity' : new_entity,
                'new_entity' : newEntity(last_thought, new_thought),
                'prompt' : prompt + new_thought + '# ',
                'last_thought' : new_thought,
                'count' : count + 1,
            }
            output_ls.append(output)
            
    if len(output_ls) == 0:
        new_thought = current_thought
        output = {
            'last_entity' : new_entity,
            'new_entity' : newEntity(last_thought, new_thought),
            'prompt' : prompt + new_thought + '# ',
            'last_thought' : new_thought,
            'count' : count + 1,
        }
        output_ls.append(output)

    return output_ls
            

cor = 0
tot = 0

for d in tqdm(dataset[0:100]):
    new_facts = set()
    
    for r in d["requested_rewrite"]:
        new_facts.add(f'{r["prompt"].format(r["subject"])} {r["target_new"]["str"]}')
        
    fact_ls = list(new_facts)
    tot += 1
    for q in d["questions"][:1]:
        state_ls = []
        initial_state = {
            'last_entity' : 'Crouching Tiger',
            'new_entity' : newEntity('No entity appears.', q),
            'prompt' : task_prompt + "\n\nQustion: " + q + '\nThoughts with New Knowledge: ',
            'last_thought' : q,
            'count' : 1,
        }
        state_ls.append(initial_state)
        ans = None
        while len(state_ls) > 0:
            state_ = state_ls[-1]
            del state_ls[-1]
            if state_['count'] >= 9:
                continue

            output = oneStep(state_['prompt'], state_['last_entity'], state_['new_entity'], state_['last_thought'], state_['count'])
            if isinstance(output, list):
                state_ls.extend(output)
            else:
                ans = output
                break
        
        if ans is None:
            continue

        # if the answer is correct
        if ans.lower() == d["new_answer"].lower() or ans.lower() in [ans_.lower() for ans_ in d["new_answer_alias"]]:
            cor += 1
            print(f'Multi-hop acc = {cor / tot} ({cor} / {tot})')
            break
        else:
            print(f'Multi-hop acc = {cor / tot} ({cor} / {tot})')
            pass

print(f'Multi-hop acc = {cor / tot} ({cor} / {tot})')

  1%|█▋                                                                                                                                                                       | 1/100 [00:02<03:46,  2.28s/it]

Multi-hop acc = 0.0 (0 / 1)


  2%|███▍                                                                                                                                                                     | 2/100 [00:05<04:42,  2.88s/it]

Multi-hop acc = 0.0 (0 / 2)


  3%|█████                                                                                                                                                                    | 3/100 [00:08<04:24,  2.72s/it]

...

Multi-hop acc = 0.7551020408163265 (74 / 98)


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 99/100 [04:45<00:02,  2.49s/it]

Multi-hop acc = 0.7575757575757576 (75 / 99)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:48<00:00,  2.88s/it]

Multi-hop acc = 0.75 (75 / 100)
Multi-hop acc = 0.75 (75 / 100)
